# 0. Make a plan!

![Science!](imgs/neil.gif "Loving science!")

## 0.1 Gather your data

In [1]:
from configparser import ConfigParser
from os.path import join
from os import pardir

In [2]:
config = ConfigParser()
config.read(join(pardir,'src','credentials.ini'))

APP_KEY = config['twitter']['app_key']
APP_SECRET = config['twitter']['app_secret']
OAUTH_TOKEN =  config['twitter']['oauth_token']
OAUTH_TOKEN_SECRET =  config['twitter']['oauth_token_secret']

In [3]:
from twitter import oauth, Twitter, TwitterHTTPError

In [4]:
auth = oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                   APP_KEY, APP_SECRET)

twitter_api = Twitter(auth=auth)
twitter_api.retry = True

*Full disclaimer: gender is non a binary issue. This is just a simplified example. If you are willing to expand this experiment, go ahead and contact me!*

In [5]:
brogrammers = ['jakevdp', 'rasbt', 'GaelVaroquaux', 'amuellerml', 'fperez_org', 
               'fpedregosa', 'ogrisel', 'dontusethiscode', 'randal_olson', 'tdhopper' ] 
sisgrammers = ['pkafei', 'LorenaABarba', 'jessicamckellar', 'heddle317', 'diana_clarke',
               'wholemilk', 'spang', 'cecilycarver', 'juliaelman', 'b0rk']

brotweets = []
for bro in brogrammers: 
    brotweets.extend(twitter_api.statuses.user_timeline(screen_name=bro, count=100))

sistweets = []
for sis in sisgrammers: 
    sistweets.extend(twitter_api.statuses.user_timeline(screen_name=sis, count=100))

## 0.1 Clean your data

In [6]:
import re

def clean_tweet(tweet):
    """ Simplest preprocess.

    Convert a tweet to lowercarse and replace URLs and @username by a generic token

    Args:
        tweet (str): Tweet to clean.

    Returns:
        str: Preprocessed tweet
    """
    
    tweet = tweet.lower()
    
    # Remove URL and replace them with a token
    URL_REGEX = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    tweet = re.sub(URL_REGEX, '<url>', tweet, flags=re.MULTILINE)
    
    # Remove usernames and replace them with a token
    tweet = re.sub("@([A-Za-z0-9_]+)", "<user>", tweet)

    # Remove repeated spaces
    tweet = re.sub(r"\s{2,}", " ", tweet)

    # If a character is repeated more than 4 time, keep only 3 repetitions.
    tweet = re.sub(r'(.)\1{4,}', r'\1\1\1', tweet)
    
    return tweet

In [7]:
import pandas as pd

dataset = []
# Gather the text 
for tweet in brotweets:
    cleaned_tweet = clean_tweet(tweet['text'])
    dataset.append({'id': tweet['id'], 'text': cleaned_tweet, 'class': 0})
for tweet in sistweets:
    cleaned_tweet = clean_tweet(tweet['text'])
    dataset.append({'id': tweet['id'], 'text': cleaned_tweet, 'class': 1})

pd_dataset = pd.DataFrame(dataset)

In [8]:
pd_dataset.head()

,class,id,text
0,0,752137929871462400,<user> great seeing you today!
1,0,751510274432131072,"""i just wanted to check that you got my previo..."
2,0,751374701344219136,rt <user>: <user> support for minor bugs may e...
3,0,751373414435328001,"could be controversial, but frankly i think it..."
4,0,751373066987573248,til that <user> plans to drop python 2 support...


## 0.2. Share your data

In [9]:
pd_dataset.to_csv('../corpora/full_dataset.csv')

In [10]:
pd_dataset[['class', 'id']].to_csv('../corpora/ep16.csv')

![Toast!](imgs/toast.gif "Let's celebrate!")

#  1. In the beginning we had a bag of words *(or maybe the set)* 

In [11]:
DATASET_PATH = "../corpora/full_dataset.csv"
pd_dataset = pd.DataFrame.from_csv(DATASET_PATH)
pd_dataset.head()

,class,id,text
0,0,752137929871462400,<user> great seeing you today!
1,0,751510274432131072,"""i just wanted to check that you got my previo..."
2,0,751374701344219136,rt <user>: <user> support for minor bugs may e...
3,0,751373414435328001,"could be controversial, but frankly i think it..."
4,0,751373066987573248,til that <user> plans to drop python 2 support...


## 1.1. Study the vocabulary

## 1.2. Bag of words

#  2. “You will be judged by the company you keep” -  Word embeddings